# Integer Optimization

Linear optimization that requires some of the variables to be integers are called Mixed Integer Problems (MIPs).


Severals ways to solve MIP problems:
- MPSolver
- CP-SAT solver
- Original CP solver

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
# Create the mip solver with the SCIP backend.
# SCIP stands for `Solving Constraint Integer Programs`
solver = pywraplp.Solver.CreateSolver("SCIP")

#### 1. Define the variables

In [3]:
infinity = solver.infinity()

# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, "x")
y = solver.IntVar(0.0, infinity, "y")

# In the previous examples, we use solver.NumVar instead of solver.IntVar.

print("Number of variables =", solver.NumVariables())

Number of variables = 2


#### 2. Define the constraints

In [4]:
# x + 7 * y <= 17.5
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5
solver.Add(x <= 3.5)

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 2


#### 3. Define the objective

In [5]:
# Maximize x + 10 * y
solver.Maximize(x + 10 * y)

#### 4. Call the solver

In [6]:
status = solver.Solve()
status

0

In [7]:
if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print("Objective value =", solver.Objective().Value())
    print("x =", x.solution_value())
    print("y =", y.solution_value())
else:
    print("The problem does not have an optimal solution")


print("\nAdvanced usage:")
print("Problem solved in %f milliseconds" % solver.wall_time())
print("Problem solved in %d iterations" % solver.iterations())
print("Problem solved in %d branch-and-bound nodes" % solver.nodes())

Solution:
Objective value = 23.0
x = 3.0
y = 2.0

Advanced usage:
Problem solved in 351.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


## Comparing the Linear and Integer optimization

We will perform the following to compare the result

- instead of creating the MIP solver with SCIP backend
    - create the linear solver with GLOP backend
- instead of declaring the integer variables `IntVar`
    - declare continuous variables `NumVar`

In [8]:
solver = pywraplp.Solver.CreateSolver("GLOP")

infinity = solver.infinity()

x = solver.NumVar(0.0, infinity, "x")
y = solver.NumVar(0.0, infinity, "y")

print("Number of variables =", solver.NumVariables())

Number of variables = 2


In [9]:
# x + 7 * y <= 17.5
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5
solver.Add(x <= 3.5)

print("Number of constraints =", solver.NumConstraints())

# Maximize x + 10 * y
solver.Maximize(x + 10 * y)

status = solver.Solve()
status

Number of constraints = 2


0

In [10]:
if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print("Objective value =", solver.Objective().Value())
    print("x =", x.solution_value())
    print("y =", y.solution_value())
else:
    print("The problem does not have an optimal solution")


print("\nAdvanced usage:")
print("Problem solved in %f milliseconds" % solver.wall_time())
print("Problem solved in %d iterations" % solver.iterations())
print("Problem solved in %d branch-and-bound nodes" % solver.nodes())

Solution:
Objective value = 25.0
x = 0.0
y = 2.5

Advanced usage:
Problem solved in 30081.000000 milliseconds
Problem solved in 0 iterations
Problem solved in -1 branch-and-bound nodes


### Using array to define a model

https://developers.google.com/optimization/mip/mip_var_array

#### 1. Create the data

In [28]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["constraint_coeffs"] = [
        [5, 7, 9, 2, 1],
        [18, 4, -9, 10, 12],
        [4, 7, 3, 8, 5],
        [5, 13, 16, 3, -7],
    ]
    data["bounds"] = [250, 285, 211, 315]
    data["obj_coeffs"] = [7, 8, 2, 9, 6]
    data["num_vars"] = 5
    data["num_constraints"] = 4
    return data

#### 2. Instantiate the data

In [29]:
data = create_data_model()

#### 3. Instantiate the solver

In [30]:
# Create the MIP solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

#### 4. Define the variables

In [31]:
infinity = solver.infinity()
x = {}
for j in range(data["num_vars"]):
    x[j] = solver.IntVar(0, infinity, f"x[{j}]")
print("Number of variables =", solver.NumVariables())

Number of variables = 5


#### 5. Define the constraints

In [32]:
for i in range(data["num_constraints"]):
    constraint = solver.RowConstraint(0, data["bounds"][i], "")
    for j in range(data["num_vars"]):
        constraint.SetCoefficient(x[j], data["constraint_coeffs"][i][j])

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 4


#### 6. Define the objective

In [33]:
objective = solver.Objective()

for j in range(data["num_vars"]):
    objective.SetCoefficient(x[j], data["obj_coeffs"][j])
objective.SetMaximization()

In [34]:
status = solver.Solve()
status

0

In [35]:
if status == pywraplp.Solver.OPTIMAL:
    print("Objective value =", solver.Objective().Value())
    for j in range(data["num_vars"]):
        print(x[j].name(), " = ", x[j].solution_value())
    print()
    print("Problem solved in %f milliseconds" % solver.wall_time())
    print("Problem solved in %d iterations" % solver.iterations())
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
else:
    print("The problem does not have an optimal solution.")

Objective value = 259.99999999999966
x[0]  =  8.0
x[1]  =  21.0
x[2]  =  0.0
x[3]  =  2.0
x[4]  =  3.0

Problem solved in 2512.000000 milliseconds
Problem solved in 71 iterations
Problem solved in 7 branch-and-bound nodes
